In [2]:
import numpy as np

In [3]:
# Input details
a=np.nan;
X=np.array([
[4, a, 3, 3, 3, 4],
[a, 1, 2, 2, 2, 2],
[3, 2, 2, a, 2, 2],
[2, a, 2, 3, 3, a],
[5, 3, 4, 4, a, 4]]);
lambda_U=1;
lambda_V=1;
K=2;

In [4]:
# Some fixed preliminary computations
N,M=X.shape

In [5]:
# Stopping criteria
Max_iter=1000;
L=np.zeros(Max_iter)
L_stop=0.0001;
# Random initialization of U and V
U=np.random.rand(N+1,K);
V=np.random.rand(M+1,K);

In [6]:
U, V

(array([[0.49179965, 0.11899864],
        [0.15133971, 0.78670997],
        [0.48959377, 0.42315159],
        [0.9723632 , 0.91799262],
        [0.94301122, 0.17679013],
        [0.85485281, 0.38620768]]),
 array([[0.47942483, 0.73922758],
        [0.00434267, 0.72548899],
        [0.44429963, 0.65845471],
        [0.39468882, 0.29926056],
        [0.83670273, 0.84506739],
        [0.72284112, 0.77497735],
        [0.94861077, 0.49281338]]))

In [7]:
vm = {}
vmvmt = {}
un = {}
ununt = {}
## Iterative loop
for iter in np.arange(0,Max_iter).reshape(-1):
    # for updating U
# --- First computing v_m * v_m^T
    for m in np.arange(0,M).reshape(-1):
        vm[m] = np.transpose(V[m,:])
        #vm.append(np.transpose(V[m,:]))
        vmvmt[m] = vm[m] * np.transpose(vm[m])
        #vmvmt.append(vm[m] * np.transpose(vm[m]))
    # --- Now computing updated u_n for each n
    for n in np.arange(0,N).reshape(-1):
        omega_r_n = np.argwhere(~np.isnan(X[n,:]) )
        #print(np.isnan(X[n,:]))
        sum_vmvmt = np.zeros((K,K))
        for m in omega_r_n.reshape(-1):
            sum_vmvmt = sum_vmvmt + vmvmt[m]
        
        first_term = sum_vmvmt + lambda_U * np.eye(K)
        first_term_det = np.linalg.det(first_term)
        first_term_inv = np.linalg.inv(first_term)
        sum_Xnmvm = np.zeros((K,1))
        for m in omega_r_n.reshape(-1):
            sum_Xnmvm = sum_Xnmvm + X[n,m] * vm[m]
        #print(first_term)
        un[n] = np.matmul(first_term_inv,sum_Xnmvm)
        #un.append(first_term_inv * sum_Xnmvm
        #un.append(np.matmul(sum_Xnmvm, first_term))
        #print(sum_Xnmvm)
        #print("U[n,:] = ",U[n,:])
        #print("un[n] = ",un[n])
        U[n,:] = un[n][0]
    # for updating V
# --- First computing v_m * v_m^T
    for n in np.arange(0,N).reshape(-1):
        un[n] = np.transpose(U[n,:])
        #un.append(np.transpose(U[n,:]))
        ununt[n] = un[n] * np.transpose(un[n])
        #ununt.append(un[n] * np.transpose(un[n]))
    # --- Now computing updated u_n for each n
    for m in np.arange(0,M).reshape(-1):
        omega_r_m = np.transpose(np.argwhere(~np.isnan(X[:,m]) ))
        sum_ununt = np.zeros((K,K))
        for n in omega_r_m.reshape(-1):
            sum_ununt = sum_ununt + ununt[n]
        first_term = sum_ununt + lambda_V * np.eye(K)
        first_term_inv = np.linalg.inv(first_term)
        sum_Xnmun = np.zeros((K,1))
        for n in omega_r_m.reshape(-1):
            sum_Xnmun = sum_Xnmun + X[n,m] * un[n]
        vm[m] = np.matmul(first_term_inv,sum_Xnmun)
        #vm.append(np.matmul(first_term_inv, sum_Xnmun))
        V[m,:] = vm[m][0]
    # Computing the loss function to see if we can stop
# Square loss for all the unknowns
    residue = 0
    for m in np.arange(0,M).reshape(-1):
        for n in np.arange(0,N).reshape(-1):
            if np.isnan(X[n,m]):
                #do nothing
                pass
            else:
                #residue = (X[n,m] - U[n,:] * np.transpose(V[m,:])) ** 2
                #print("X[n,m] = ",X[n,m])
                #print("U[n,:] = ", U[n,:])
                #print("V[m,:] = ", V[m,:])
                residue = ((X[n,m]-np.matmul(U[n,:],V[m,:].T)))**2;
    print(residue)
    L[iter] = residue
    if L[iter] < L_stop:
        print('Termination at ' +str(iter)+' iteration')
        break


0.0339628942588178
0.0044841837748486255
0.0010152159671792049
5.6484768416199936e-05
Termination at 3 iteration
